In [ ]:
%matplotlib inline
import os
import pandas
from scipy.stats import f_oneway
from statsmodels.stats.multitest import multipletests
import numpy
import math
import itertools 
from scipy.stats import ttest_ind
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from scipy import stats
from sklearn import preprocessing
import seaborn as sns

##### Представление данных

In [ ]:
mir_ratio_column = "miR_Ratio"
mir_ratio_2_column = "2_miR Ratio"
hemscore_column = "Hem_Score"
gender_column = "Gender"
diabetes_column = "Diabetes"
age_column = "Age"

##### Обработка данных

In [ ]:
class SignificantMicroRNAFinder(object):

    def __init__(
            self, path, mir_ratio_2_theshold=40000, 
            housekeeping_gene="Ct_hsa-miR-16-5p", 
            special_housekeeping_gene=None,
            special_miRNA=None,
            output_files_folder="results/temp",
            output_files_prefix="mir16_hk_",
    ):
        self.path = path
        self.mir_ratio_2_theshold = mir_ratio_2_theshold
        self.housekeeping_gene = housekeeping_gene
        self.special_housekeeping_gene = special_housekeeping_gene
        self.special_miRNA = special_miRNA
        
        self.output_files_folder = output_files_folder
        self.output_files_prefix = output_files_prefix
        if not os.path.exists(self.output_files_folder):
            os.makedirs(self.output_files_folder)
        self.output_prefix = os.path.join(self.output_files_folder, self.output_files_prefix)
        
        self.data = self.prepare_data()
        self.data = self.normalize_expression(self.data)

        
    def normalize_expression(self, df):
        self.norm_exp_cols = []
        exp_cols = [col for col in df.columns if "Ct_" in col]
        for col in exp_cols:
            df[f"{col}_normalized"] = df.apply(
                lambda x: 
                    2**x[self.housekeeping_gene]/2**x[col]
                    if self.special_miRNA is None 
                        or col not in self.special_miRNA
                        or self.special_housekeeping_gene is None 
                        or pandas.isnull(x[self.special_housekeeping_gene])
                    else 2**x[self.special_housekeeping_gene]/2**x[col], 
                axis=1
            )
            self.norm_exp_cols.append(f"{col}_normalized")

        return df

    def prepare_data(self):
        data = pandas.read_excel(self.path)
        data = data.replace(["Undetermined"], [None])
        data[mir_ratio_2_column] = data[mir_ratio_column].map(lambda x: 2**x)

        data.plot.scatter(x=hemscore_column, y=mir_ratio_2_column, title="Before filtering")

        data = self.filter_samples(data)
        data.plot.scatter(x=hemscore_column, y=mir_ratio_2_column, title="After filtering")

        return data


    def filter_samples(self, data):
        return data[data[mir_ratio_2_column] < self.mir_ratio_2_theshold]

    def regression_with_confounding_factors(
        self, group_column="Group",
        confounding_factors=[
            hemscore_column,
            gender_column,
            diabetes_column,
            age_column
        ]
    ):
        le = preprocessing.LabelEncoder()
        
        data = self.data.copy(deep=True)
        
        potential_microRNAs = set(self.norm_exp_cols) - set([self.housekeeping_gene + "_normalized"])
        if self.special_housekeeping_gene:
            potential_microRNAs -= set([self.special_housekeeping_gene + "_normalized"])
        
        output_suffix = "_%s.xlsx" % "_".join(confounding_factors)

        # Create linear regression object
        lr_scores = pandas.DataFrame()
        regr = linear_model.LinearRegression()
        log2ratio = pandas.DataFrame()
        for group1, group2 in itertools.combinations(data[group_column].dropna().unique().tolist(), 2):
            comparison_name = f"{group1}_vs_{group2}"
            res = {}
            l2r = {}
            for variable in list(potential_microRNAs):
                # Train the model using the training sets
                subdf = data[data[group_column].isin([group1, group2])].dropna(subset=confounding_factors + [variable])
                X = subdf[list(set([variable] + confounding_factors))]
                y = le.fit(subdf[group_column]).transform(subdf[group_column])

                X2 = sm.add_constant(X)
                est = sm.OLS(y, X2)
                est2 = est.fit()
                print("\n\n\nComparing groups", group1, group2, "for vairable", variable, "\n\n\n")
                print(est2.summary())
                res[variable] = est2.pvalues[variable]
                l2r[variable] = math.log(
                    X.loc[subdf[subdf[group_column] == group1].index][variable].mean() / 
                    X.loc[subdf[subdf[group_column] == group2].index][variable].mean(),
                    2
                )
            log2ratio = log2ratio.append(pandas.DataFrame(l2r, index=[comparison_name]))
            lr_scores = lr_scores.append(pandas.DataFrame(res, index=[comparison_name]))
            multitests = multipletests(
                list(lr_scores.T[comparison_name].values), 
                alpha=0.05, 
                method="fdr_bh",
            )
            lr_scores.loc[comparison_name + "_mt"] = dict(zip(lr_scores.columns, multitests[1]))
        
        lr_scores.to_excel(f"{self.output_prefix}_group_col_{group_column}_significance_{output_suffix}")
        log2ratio.to_excel(f"{self.output_prefix}_group_col_{group_column}_expression_log2Ratio_{output_suffix}")

        return lr_scores, log2ratio

##### Поиска значимо изменяющихся микроРНК в miR_CAD_Database

In [ ]:
finder = SignificantMicroRNAFinder(
    "data/miR_CAD_Database_2019.xlsx", 
    special_housekeeping_gene="Ct_hsa-miR-16-5p_new",
    special_miRNA = [
        "Ct_hsa-miR-146a-5p",
        "Ct_hsa-miR-375", 
        "Ct_hsa-miR-23a-3p", 
        "Ct_miR-451a"
    ],
    output_files_folder="results/miR_AF_Database_2019"
)
finder.regression_with_confounding_factors()

##### Поиск значимо изменяющихся микроРНК в AF_Database

In [ ]:
finder = SignificantMicroRNAFinder(
    "data/miR_AF_Database_2019.xlsx", 
    output_files_folder="results/miR_AF_Database_2019"
)
finder.regression_with_confounding_factors()

##### Поиска значимо изменяющихся микроРНК в зависимости от различных рисковых коэффициентов и шкал

In [ ]:
finder = SignificantMicroRNAFinder(
    "data/miR_AF_Database_2019_scales.xlsx", 
    output_files_folder="results/miR_AF_Database_2019_scales"
)

In [ ]:
finder.regression_with_confounding_factors()
finder.regression_with_confounding_factors(group_column="CHA2DS2VASc_1")
finder.regression_with_confounding_factors(group_column="CHA2DS2VASc_2")
finder.regression_with_confounding_factors(group_column="SCORE_5")
finder.regression_with_confounding_factors(group_column="ACC_AHA_7,5")